# Carregamento do dataset

### Informações gravadas em: https://github.com/bneurd/opt020

In [1]:
from scipy.io import loadmat

f = loadmat("../multi/Sub1_1_multitarget.mat")
print(f['Data'][0][0][8][0])
print(f['Data'][0][0][7].shape)
data = f['Data'][0][0][7]
labels = f['Data'][0][0][8][0]

[7.5 8.2 7.  8.2 6.  7.5 6.  6.  8.2 8.2]
(8192, 10)


Dado o exemplo acima, faça o carregamento dos cinco participantes, contatenando suas seções ou *trials*. Basta em um `for` externo aumentar a dimensionalidade para a quantidade de participantes e em um `for` interno, concatenar as seções com dez (10) *targets* cada uma.

Ao final, teremos um *shape* igual a `(5, 8192, 20)`.

### Utilizando a biblioteca MNE

Nesta etapa, vamos criar um objeto MNE para armazenar os dados juntamente com os labels.

Para este tipo de tarefa, é recomendo o uso do objeto Epochs, pois armazena tanto os dados quanto os metadados.

In [2]:
# inicialmente criamos o objeto info
import mne

n_channels = 1
sfreq = 512
ch_names = ['Oz']
ch_types = ['eeg']
info = mne.create_info(ch_names, sfreq=sfreq, ch_types=ch_types)
info

<Info | 7 non-empty values
 bads: []
 ch_names: Oz
 chs: 1 EEG
 custom_ref_applied: False
 highpass: 0.0 Hz
 lowpass: 256.0 Hz
 meas_date: unspecified
 nchan: 1
 projs: []
 sfreq: 512.0 Hz
>

In [3]:
import numpy as np
from sklearn.preprocessing import LabelEncoder

# Shape dos dados: (n_epochs, n_channels, n_times) => (10, 1, 8192)
data_correct = data.swapaxes(1, 0)
data_correct = np.expand_dims(data_correct, axis=1)
# método para transformar labels categóricos
le = LabelEncoder()
events = np.column_stack((
    np.array(range(len(labels))),
    np.zeros(labels.shape[0], dtype=int),
    le.fit_transform(labels))
)
# organizar os targets de acordo com todos os diferentes valores de labels
event_dict = {'6':0, '7':1, '7.5':2, '8.2':3}
mne_data = mne.EpochsArray(data_correct, info, events, event_id=event_dict)
mne_data.save('ssvep-epo.fif', overwrite=True)

Not setting metadata
10 matching events found
No baseline correction applied
0 projection items activated
Overwriting existing file.
Overwriting existing file.
